In [1]:
import pandas as pd
import numpy as np
import statistics as stat

In [2]:
data = pd.read_csv('scdb-20rfq-60cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
len(data)

8756

In [4]:
#grouping every transaction by its ID
grouped_data = data.groupby('Tx_id')

In [5]:
#return_accept_id_group = data.groupby('Accept_id')

In [6]:
tx_id = list(grouped_data.groups.keys())
len(tx_id)

1538

In [7]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])
rfq_count = 0
bid_count = 0
create_count = 0
accept_count = 0
return_count_1 = 0
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'REQUEST_FOR_QUOTE':
        rfq_count += 1
    if data_unique_ids.iloc[i]['Tx_type'] == 'CREATE':
        create_count += 1    
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_count += 1   
    if data_unique_ids.iloc[i]['Tx_type'] == 'ACCEPT':
        accept_count += 1 
    if data_unique_ids.iloc[i]['Tx_type'] == 'RETURN':
        return_count_1 += 1   
print(f"#RFQs {rfq_count}")
print(f"#BIDs =  {bid_count}")
print(f"#CREATEs =  {create_count}")
print(f"#ACCEPTs =  {accept_count}")
print(f"#RETURNs =  {return_count_1}")
print(f"#Total =  {return_count_1 + rfq_count + bid_count + create_count + accept_count}")

#RFQs 20
#BIDs =  499
#CREATEs =  500
#ACCEPTs =  20
#RETURNs =  499
#Total =  1538


In [8]:
# Time per each transaction type
time_all_committed_bids = {}
time_passed_bids = []
bid_time = {}
bids_overfit = []
bids_underfit = []
other_bids = []

time_all_committed_creates = {}
time_passed_creates = []
create_time = {}
creates_overfit = []
creates_underfit = []
other_creates = []

time_all_committed_rfqs = {}
time_passed_rfqs = []
rfq_time = {}
rfq_overfit = []
rfq_underfit = []
other_rfqs = []

time_all_committed_accepts = {}
time_passed_accepts = []
accept_time = {}
time_failed_accepts = []
accept_underfit = []
accept_overfit = []
other_accepts = []


time_all_committed_returns = {}
time_passed_returns= []
return_time = {}
return_overfit = []
return_underfit = []
other_returns = []


time_bids = []
time_id_bid = {}


time_rfqs = []
time_id_rfqs = {}

time_accept = []
time_id_accept = {}

time_create = []
time_id_create = {}

time_returns = []
accept_return_count = {}


time_id_return = {}
for i in range(0, len(tx_id)):
    temp_list = []
    return_count = 0
#     if "REQUEST_FOR_QUOTE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list)
#         time_rfqs.append(actual_time)
#         time_id_rfqs[tx_id[i]] = actual_time

    if "REQUEST_FOR_QUOTE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            rfq_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(tx_id[i])
            rfq_underfit.append(temp_list)
        elif len(temp_list) <= 10:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            rfq_overfit.append(temp_list)
            
        else:
            temp_list.append(tx_id[i])
            other_rfqs.append(temp_list)
        
        
#     if "CREATE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list) 
#         time_create.append(actual_time)
#         time_id_create[tx_id[i]] = actual_time

    if "CREATE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            create_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(tx_id[i])
            creates_underfit.append(temp_list)
        elif len(temp_list) <= 10:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_passed_creates.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            creates_overfit.append(temp_list)
            
        else:
            temp_list.append(tx_id[i])
            other_creates.append(temp_list)
    """ 
        if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0] or 'None' != grouped_data.get_group(tx_id[i])["Accept_id"].to_list()[0]:
        print(i)
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        for j in range(0, len(data)):
            if data['Event'][j] = "RETURN" and tx_id[i] == data['Accept_id'][j]:
                temp_list.append(data['Time'][j])
        print(temp_list)
        actual_time = max(temp_list)
        time_accept.append(actual_time)
        """
    
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        for j in range(0, len(data)):
            if data['Event'][j] == "commit_tx" and tx_id[i] == data['Accept_id'][j]:
                temp_list.append(data["Time"][j])
                return_count += 1
        accept_return_count [tx_id[i]] = return_count
        last_return = max(temp_list)
        time_accept.append(last_return)
        time_id_accept[tx_id[i]] = last_return
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            temp_list.append(last_return)
            actual_time = max(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            accept_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(last_return)
            temp_list.append(tx_id[i])
            accept_underfit.append(temp_list)
        elif len(temp_list) <= 10:
            time_all_committed_accepts[tx_id[i]] = actual_time
            temp_list.append(last_return)
            actual_time = max(temp_list)
            time_passed_accepts.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            accept_overfit.append(temp_list)
            
        else:
            temp_list.append(last_return)
            temp_list.append(tx_id[i])
            other_accepts.append(temp_list)

    
#     if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         for j in range(0, len(data)):
#             if data['Event'][j] == "commit_tx" and tx_id[i] == data['Accept_id'][j]:
#                 temp_list.append(data["Time"][j])
#                 return_count += 1
#         accept_return_count [tx_id[i]] = return_count
#         actual_time = max(temp_list)
#         time_accept.append(actual_time)
#         time_id_accept[tx_id[i]] = actual_time
         
    if "RETURN" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) >= 4:
            time_all_committed_returns[tx_id[i]] = temp_list
            actual_time = max(temp_list)
            time_passed_returns.append(actual_time)
            return_time[tx_id[i]] = actual_time
        elif len(temp_list) < 4:
            temp_list.append(tx_id[i])
            return_underfit.append(temp_list)
        elif len(temp_list) > 7:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_passed_returns.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            return_overfit.append(temp_list)
            
        else:
            temp_list.append(tx_id[i])
            other_returns.append(temp_list)
    
#     if "RETURN" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list)
#         time_returns.append(actual_time)
#         time_id_return[tx_id[i]] = actual_time

            
    if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            time_all_committed_bids[tx_id[i]] = temp_list
            actual_time = max(temp_list)
            time_passed_bids.append(actual_time)
            bid_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(tx_id[i])
            bids_underfit.append(temp_list)
        elif len(temp_list) == 7:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_passed_bids.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            bids_overfit.append(temp_list)
            
        else:
            other_bids.append(temp_list)
#     if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list)
#         if actual_time != 0:
#             time_bids.append(actual_time)
#             time_id_bid[tx_id[i]] = actual_time

In [9]:
time_all_committed_rfqs = {}
time_passed_rfqs = []
rfq_time = {}
rfq_overfit = []
rfq_underfit = []
not_validated_rfqs = []
passed_validation = {}
for i in range(0, len(tx_id)):
    temp_list = []
    if "REQUEST_FOR_QUOTE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            rfq_time[tx_id[i]] = actual_time
            if grouped_data.get_group(tx_id[i])["Event"].to_list()[1] == "before_tendermint":
                val = temp_list[1] - temp_list[0]
                passed_validation[tx_id[i]] = val
        elif  1 < len(temp_list) < 3:
            val = temp_list[1] - temp_list[0]
            passed_validation[tx_id[i]] = val
            temp_list.append(tx_id[i])
            rfq_underfit.append(temp_list)
        elif  6 < len(temp_list) <= 10:
            val = temp_list[1] - temp_list[0]
            passed_validation[tx_id[i]] = val
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            rfq_overfit.append(temp_list)
            
            
        else:
            temp_list.append(tx_id[i])
            not_validated_rfqs.append(temp_list)

In [10]:
len(time_id_accept)

20

In [11]:
accept_return_count

{'01a8c04001b8a2b130c3466dca1e55c68b683aa8f3f55e2af062f0351b33b216': 25,
 '19f6ece27dce7909ecf314dbbf25ea83d097184eb9b4d5a9b0fbe2c642c96433': 25,
 '2599a2aa663dde584091f55c6873d0dad4d72740cef22a98ff5bfd1de1d37262': 25,
 '337c781bf1b00b1788f379860be0f548c62dac8795ab9ca238390f81f7528d2c': 25,
 '3e1b95d5f516967275d039023fa87d77a4550243eaf94651f227326b8ab13c4e': 25,
 '517d8cb41c5f51439e9dc3f31d8391ab6da0ceb46098037a4f4315cb37f22e3f': 25,
 '5ade1fb7b29bfb09f0e1627d307ab8ebdd0aa459c54da0542f798f2d0c9fe9a6': 25,
 '60611b0f9b3f79b11e6526b21ad8b971d5f3c834902cb34bacf9d6e68301bca9': 25,
 '6eb117fba67ce0824d20ad01e0092e6b6ae29d8352f90a95528213721311a7b2': 24,
 '6f8373907216cf129867e4cc4a5468929ee7803b3f5a68870f1e10c024ee4737': 25,
 '82c17dd6ba9c0342bf104fc126a0c5dae8f49579499e47e334d46096470a8072': 25,
 '864c3f08cd170291686f1bc791da3c1436928b3f16da5da876d9eb5fdc20a3db': 25,
 '92f3111d0b7ea2e62804343583cf5ba713b94d1b23c169504c3be01eaf51689c': 25,
 '97ac4af36d096aaa1cd2f437581805897b21ad387521153fe

In [12]:
df_passed_creates = pd.read_csv('create_data.csv')
df_passed_bids = pd.read_csv('bid_data.csv')
df_passed_accepts = pd.read_csv('accept_data.csv')
df_passed_returns = pd.read_csv('return_data.csv')
df_passed_rfqs = pd.read_csv('rfq_data1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'create_data.csv'

In [13]:
time_passed_creates = df_passed_creates['0'].to_list()
time_passed_bids = df_passed_bids['valid_BIDs'].to_list()
time_passed_accpets = df_passed_accepts['0'].to_list()
time_passed_returns = df_passed_returns['valid_RETURNs'].to_list()
time_passed_rfqs = df_passed_rfqs['time'].to_list()

NameError: name 'df_passed_creates' is not defined

In [14]:
# Analysis of CREATE Transaction
time_passed_creates.remove(max(time_passed_creates)) 
create_mean = "{:.2f}".format(stat.mean(time_passed_creates))
create_median = stat.median(time_passed_creates)
create_max = max(time_passed_creates)
create_min = min(time_passed_creates)
create_std_dev = "{:.2f}".format(stat.stdev(time_passed_creates))

print(create_mean)
print(create_median)
print(create_max)
print(create_min)
print(create_std_dev)
print(f"total creates = {len(time_passed_creates)}")

create_dict = {
               'min': create_min,
               'max': create_max,
               'average': create_mean,
               'median' : create_median,
               'standard_deviation' : create_std_dev,
               '# of txs' : len(time_passed_creates)}

190.27
179
243
171
17.74
total creates = 499


In [15]:
# Analysis of RFQ Transaction
if len(time_passed_rfqs) < 2:
    time_rfqs.append(time_passed_rfqs[0])
rfq_mean = "{:.2f}".format(stat.mean(time_passed_rfqs))
rfq_median = stat.median(time_passed_rfqs)
rfq_max = max(time_passed_rfqs)
rfq_min = min(time_passed_rfqs)
rfq_std_dev = "{:.2f}".format(stat.mean(time_passed_rfqs))

print(rfq_mean)
print(rfq_median)
print(rfq_max)
print(rfq_min)
print(rfq_std_dev)
print(f"total RFQs = {len(time_passed_rfqs)}")

rfq_dict = {
               'min': rfq_min,
               'max': rfq_max,
               'median' : rfq_median,
               'average': rfq_mean,    
               'standard_deviation' : rfq_std_dev,
               '# of txs' : rfq_count}

571.60
586.5
793
323
571.60
total RFQs = 20


In [16]:
# Analysis of BID Transaction 
# only passed BIDs 
bid_mean = "{:.2f}".format(stat.mean(time_passed_bids))
bid_median = stat.median(time_passed_bids)
bid_max = max(time_passed_bids)
bid_min = min(time_passed_bids)
bid_std_dev = "{:.2f}".format(stat.stdev(time_passed_bids))


print(f" bid_mean =  {bid_mean} \n bid_median = {bid_median} \n bid_max = {bid_max} \n"
     f" bid_std_dev = {bid_std_dev} \n total_bids = {len(time_passed_bids)}")

bid_dict = {
               'min': bid_min,
               'max': bid_max,
               'average': bid_mean,
               'median' : bid_median,
               'standard_deviation' : bid_std_dev,
               '# of txs' : bid_count
              }

 bid_mean =  224.98 
 bid_median = 197 
 bid_max = 1040 
 bid_std_dev = 87.61 
 total_bids = 499


In [17]:
# Analysis of ACCEPT Transaction

accept_mean =  "{:.2f}".format(stat.mean(time_passed_accepts))
accept_median = stat.median(time_passed_accepts)
accept_max = max(time_passed_accepts)
accept_min = min(time_passed_accepts)
accept_std_dev = "{:.2f}".format(stat.stdev(time_passed_accepts))

print(accept_mean)
print(accept_median)
print(accept_max)
print(accept_min)
print(accept_std_dev)
print(f"total ACCEPTs = {len(time_passed_accepts)}")

accept_dict = {
               'min': accept_min,
               'max': accept_max,
               'median' : accept_median,
               'average': accept_mean,
               'standard_deviation' : accept_std_dev,
               '# of txs' : accept_count}

2630.00
2627.5
3406
1933
367.81
total ACCEPTs = 20


In [18]:
# Analysis of RETURN Transaction

return_mean = "{:.2f}".format(stat.mean(time_passed_returns))
return_median = stat.median(time_passed_returns)
return_max = max(time_passed_returns)
return_min = min(time_passed_returns)
return_std_dev =  "{:.2f}".format(stat.stdev(time_passed_returns))

print(return_mean)
print(return_median)
print(return_max)
print(return_min)
print(return_std_dev)
print(f"total RETURNs = {len(time_passed_returns)}")
return_dict = {
               'min': return_min,
               'max': return_max,
               'median' : return_median,
               'average': return_mean,
               'standard_deviation' : return_std_dev,
               '# of txs' : len(time_passed_returns)}

2568.52
2505
3406
1723
369.73
total RETURNs = 499


In [19]:
list_of_dict = [create_dict, rfq_dict, bid_dict, accept_dict, return_dict]
results = pd.DataFrame(list_of_dict, index = ['CREATE','RFQ', 'BID', 'ACCEPT', 'RETURN'])

In [20]:
results

,min,max,average,median,standard_deviation,# of txs
CREATE,171,243,190.27,179.0,17.74,499
RFQ,323,793,571.60,586.5,571.60,20
BID,184,1040,224.98,197.0,87.61,499
ACCEPT,1933,3406,2630.00,2627.5,367.81,20
RETURN,1723,3406,2568.52,2505.0,369.73,499


In [73]:
successfull_rate_accept = len(time_passed_accepts) / accept_count * 100
successfull_rate_bid = len(time_passed_bids) / bid_count * 100
successfull_rate_create = len(time_passed_creates) / create_count * 100
successfull_rate_return = len(time_passed_returns) / return_count_1 * 100
successfull_rate_rfq = len(time_passed_rfqs) / rfq_count * 100

print(f"rfq rate: {successfull_rate_rfq} %")
print(f"create rate: {successfull_rate_create} %")
print(f"bid rate: {successfull_rate_bid} %")
print(f"accept rate: {successfull_rate_accept} %")
print(f"return rate: {successfull_rate_return} %")


rfq rate: 100.0 %
create rate: 99.8 %
bid rate: 100.0 %
accept rate: 100.0 %
return rate: 100.0 %
